In [1]:
'''
测试
pandas的json读取
'''
import pandas as pd
info_filename = r"E:\pycharm files\爬虫\房天下二手房定价模型\data\data3000.json"
df = pd.read_json(info_filename)

In [8]:
# 查看物业类型属性
print(set(df['物业类型']))

df1 = df.iloc[,:]

{'别墅', '住宅'}


In [88]:
# 查看多少属性后是别墅属性，删除别墅属性
import numpy as np

df.info()
df1 = df.iloc[:,0:37]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985 entries, 0 to 2984
Data columns (total 49 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   url         2985 non-null   object 
 1   标题          2985 non-null   object 
 2   地址          2985 non-null   object 
 3   优势1         2985 non-null   object 
 4   优势2         2985 non-null   object 
 5   优势3         2985 non-null   object 
 6   总价          2985 non-null   float64
 7   户型          2985 non-null   object 
 8   建筑面积        2985 non-null   object 
 9   单价          2985 non-null   object 
 10  朝向          2975 non-null   object 
 11  装修          2975 non-null   object 
 12  楼层层次        2975 non-null   object 
 13  总楼层         2975 non-null   float64
 14  ID          2985 non-null   int64  
 15  小区          2985 non-null   object 
 16  区域          2985 non-null   object 
 17  有无电梯        2975 non-null   object 
 18  产权性质        2960 non-null   object 
 19  住宅类别        1228 non-null  

In [163]:
'''
处理户型属性：
将四室两厅两卫 拆分为 卧室4 客厅2 卫生间2
'''
import re
record_size = len(df1)
zeros_array = np.zeros((record_size,3))
# 户型df
df_houseType = pd.DataFrame(zeros_array,columns=['卧室','客厅','卫生间'])

# map函数
def split_houseType(x):
    numbers = re.findall(r'\d+', x)
    if(numbers==[]):
        return [0,0,0]
    return numbers

series_houseType = df1["户型"].map(lambda x:split_houseType(x))
for i in range(record_size):
    df_houseType.iloc[i,0] = int(series_houseType[i][0])
    df_houseType.iloc[i,1] = int(series_houseType[i][1])
    df_houseType.iloc[i,2] = int(series_houseType[i][2])

df2 = pd.concat([df1,df_houseType],axis=1)

In [164]:
# value_counts()`：计算Series中每个值的出现次数。
print(df2['容  积  率'].isna().sum())
print(df2['绿  化  率'].isna().sum())
print(df2['物业费用'].isna().sum())
print(df2['产权年限'].isna().sum())
print(df2['人车分流'].isna().sum())


21
21
93
1328
0


In [165]:
# 产权年限提取数字
df2['产权年限'] = df2['产权年限'].fillna('70')
df2['产权年限'] = df2['产权年限'].map(lambda x:int(x[0:2]))

In [166]:
# 人车分流转为0-1
dic_map = {'无':0,'有':1}
df2['人车分流'] = df2['人车分流'].map(dic_map)

In [167]:
def clean_green(x):
    numbers = re.findall(r'\d+', x)
    return int(numbers[0])
print(df2['绿  化  率'].value_counts()[0])
idx_nan = df2['绿  化  率'].isna()
df2['绿  化  率'] = df2['绿  化  率'].fillna("30%")
df2['绿  化  率'] = df2['绿  化  率'].map(lambda x:clean_green(x))

30%       930
35%       349
40%       225
20%       161
45%        98
         ... 
31.2%       1
40.1%       1
32.17%      1
41.8%       1
30.2%       1
Name: 绿  化  率, Length: 97, dtype: int64


In [176]:

print(df2['绿  化  率'].value_counts().index[0])

30
